In [40]:
import torch
dev=torch.device('cpu')
centroids=torch.load('/home/jzgrp/xiaomenghan/vptq_qwen2-7/model.layers.0.self_attn.q_proj_centroids.pt').to(dev)
index=torch.load('/home/jzgrp/xiaomenghan/vptq_qwen2-7/model.layers.0.self_attn.q_proj_indices.pt').to(dev)
print(index.shape)
centroids=centroids.weight.view(1,65536,8)
high_mask = 0xffff0000  # 未使用，可保留或删除
low_mask = 0x0000ffff   # 未使用，可保留或删除
new_index = torch.zeros([448, 3584], dtype=torch.int32, device=dev)
lower = index[0] & 0xFFFF              # 提取低16位，形状 (448, 1792)
upper = (index[0] >> 16) & 0xFFFF      # 提取高16位，形状 (448, 1792)
new_index = torch.stack([lower, upper], dim=2).view(448, 3584)  # 重组为 (448, 3584)
centroids = centroids.view(65536, 8).to(dev)  # 修正拼写错误并准备张量
print(centroids[52581])
recon_weight = torch.zeros([448, 3584], device=dev)
assert new_index.min() >= 0 and new_index.max() < 65536, "new_index 包含越界索引"
# 使用矢量化操作替代嵌套循环
recon_weight = centroids[new_index, 0]
centroids = centroids.view(16384, 32).to(device=dev, dtype=torch.float32)
# 创建 code 张量，确保在 dev 上并为 float32
code = torch.zeros([16384, 4], dtype=torch.float32, device=dev)
code = centroids[:, [0, 8, 16, 24]]


/tmp/ipykernel_1605063/2397418420.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  centroids=torch.load('/home/jzgrp/xiaomenghan/vptq_qwen2-7/model.layers.0.self_attn.q_p

torch.Size([1, 448, 1792])
tensor([-0.1514,  0.2871,  0.1934, -0.9297, -0.4824, -1.0703, -0.1309, -0.1436],
       dtype=torch.float16, grad_fn=<SelectBackward0>)


In [22]:
#特征提取法
from collections import Counter
import numpy as np
entry_character=torch.zeros([65536,448])
cindex=new_index.cpu()
for i in range(new_index.shape[0]):
    counts = torch.bincount(cindex[i, :], minlength=entry_character.shape[0])
    entry_character[:,i]=counts
# print(entry_character)
    # print(counts.shape)
    # entry_character[]
# for i in range(new_index.shape[0]):
#     element_counts = Counter(cindex[i,:])
#     for j in range(entry_character.shape[0]):
#         entry_character[j,i]=element_counts[j]



tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [45]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
data=entry_character.cpu().numpy()
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# best_k = int(input("根据图形输入最佳K值: "))  # 交互式选择
# 或者直接指定 best_k = 8

kmeans = MiniBatchKMeans(n_clusters=16,
                        batch_size=8192,
                        random_state=42)
cluster_labels = kmeans.fit_predict(data_scaled)

# 5. 保存结果
np.save('cluster_labels.npy', cluster_labels)
print("聚类完成，标签已保存为cluster_labels.npy")

# 可选：查看聚类中心
print("聚类中心形状:", kmeans.cluster_centers_.shape)

In [44]:
centroids = centroids.view(65536, 8).to(dev)  # 修正拼写错误并准备张量
sorted_indices = np.argsort(cluster_labels)
centroids = centroids[sorted_indices]
centroids = centroids.view(16384, 32).to(device=dev, dtype=torch.float32)
# 创建 code 张量，确保在 dev 上并为 float32
code = torch.zeros([16384, 4], dtype=torch.float32, device=dev)
code = centroids[:, [0, 8, 16, 24]]
print(code,sorted_indices)
print(cluster_labels[52581])

tensor([[ 0.2969,  0.8789,  0.5352,  0.4922],
        [-0.2041, -0.7383, -0.4414,  1.6250],
        [ 0.8359,  0.6602,  0.3691, -0.4316],
        ...,
        [ 1.9688,  0.6250,  1.3906,  0.6289],
        [-1.3203, -0.3477,  1.3906, -1.3281],
        [ 0.2480,  0.5156,  0.6992, -0.3066]], grad_fn=<IndexBackward0>) [52581 52642 14430 ... 34454 34442 65520]
0


In [7]:
#图优化：
import networkx as nx
G = nx.Graph()
for i in range(65536):
    G.add_node(i)

# for i in range(65536):
#     for j in range (65536):
#         G.add_edge(i,j,weight=0)

# for i in range(64):
#     for j in range(64):
#        node1=new_index[i,j].item()
#        node2=new_index[i,j+1].item()
#     #    print(node1,node2)
#        if G.has_edge(node1, node2):
#             weight=G[node1][node2]['weight']+1
#             G.add_edge(node1,node2,weight=weight)
#        else:
#             G.add_edge(node1,node2,weight=1)
print("图中的顶点数：", G.number_of_nodes())
print("图中的边数：", G.number_of_edges())
import numpy as np
matrix = np.array([[1, 1, 3],
                   [1, 1, 6],
                   [7, 8, 9]])
a=set(matrix[0])
b=set(matrix[1])
similarity = a.intersection(b)
print(similarity)
index=new_index.numpy()
similarity_matrix = np.zeros((448, 448))
for i in range(448):
    for j in range(i+1, 448):
        set_i = set(index[i]) # 第i行的唯一条目集合
        set_j = set(index[j])  # 第j行的唯一条目集合
        similarity = len(set_i.intersection(set_j))  # 计算共享条目数
        similarity_matrix[i][j] = similarity
        similarity_matrix[j][i] = similarity
        print(i,j,similarity_matrix[i][j])
        def find_max_weight_edge(G, node):
    max_weight = -1
    max_edge = None
    for neighbor in G.neighbors(node):
        weight = G[node][neighbor]['weight']
        if weight > max_weight:
            max_weight = weight
            max_edge = (node, neighbor)
    return max_edge, max_weight

# 指定的顶点
specified_node = 59581
for i in range(65536) :
    edges = G.edges(i, data=True)

# 打印与节点1920相连的所有节点及其边的权重
    print(f"与节点 {i} 相连的所有节点及其边的权重：")
for edge in edges:
    print(f"节点 {edge[1]}，权重 {edge[2]['weight']}")
    # 找到与指定顶点相连的最大权重边
    # max_edge, max_weight = find_max_weight_edge(G, i)

    # if max_edge is not None:
    #     print(f"\n与顶点 {i} 相连的最大权重边为：{max_edge}，权重为：{max_weight}")
    # else:
    #     print(f"\n顶点 {i} 没有相连的边")

    import networkx as nx
import random
from collections import deque



# BFS遍历图
def bfs_traversal(G, start_node):
    visited = set()  # 记录已访问的节点
    queue = deque([start_node])
    visited.add(start_node)
    traversal_order = []  # 记录遍历顺序

    while queue:
        node = queue.popleft()
        traversal_order.append(node)
        for neighbor in G.neighbors(node):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    return traversal_order

# 从节点0开始BFS遍历
start_node = 0
traversal_result = bfs_traversal(G, start_node)

# 打印遍历结果
print("\nBFS遍历顺序（从节点 {} 开始）：".format(start_node))
print(traversal_result)

# 如果图不连通，遍历未访问的节点
if len(traversal_result) < len(G.nodes()):
    print("\n图不连通，继续遍历其他未访问的节点：")
    all_nodes = set(G.nodes())
    remaining_nodes = all_nodes - set(traversal_result)
    for node in remaining_nodes:
        if node not in visited:
            sub_traversal = bfs_traversal(G, node)
            traversal_result.extend(sub_traversal)
            print("从节点 {} 开始的遍历结果：{}".format(node, sub_traversal))

print("\n完整的BFS遍历顺序：")
print(traversal_result)

图中的顶点数： 65536
图中的边数： 0


In [9]:
import numpy
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
cindex=new_index.cpu().numpy()
# element_counts = Counter(cindex[7,:])
# print(element_counts) # 输出: Counter({3: 3, 1: 2, 2: 2})
# element_counts = Counter(cindex[8,:])
# print(element_counts) # 输出: Counter({3: 3, 1: 2, 2: 2})
# centroids = centroids.view(65536, 8).to(dev)  # 修正拼写错误并准备张量
for i in range(64):
    element_counts = Counter(cindex[i,:])
    elements = []
    frequencies = []
    print(element_counts)
    # for element, count in element_counts.most_common():
    #     elements.append(element)
    #     frequencies.append(count)
    # if frequencies[0]>=4:
        # print(element_counts)
        # positions = []
        # for i, row in enumerate(cindex):
        #     for j, value in enumerate(row):
        #         if value == elements[0]:
        #             positions.append((i, j))
        #             # print(cindex[i,j+1])
        # print(f"值为{elements[0]}的元素出现的位置:", positions)
    # 

值为14750的元素出现的位置: [(1, 2149), (1, 2281), (1, 2594), (1, 2868), (2, 3526), (9, 2415)]
值为25986的元素出现的位置: [(4, 2420), (5, 1044), (5, 1131), (5, 1189), (5, 1446), (12, 3434), (45, 785)]
值为7921的元素出现的位置: [(12, 36), (12, 295), (12, 1093), (12, 3064), (17, 495), (24, 156), (38, 403)]
值为19530的元素出现的位置: [(13, 1035), (13, 1942), (13, 1969), (13, 2046), (28, 2491), (37, 2673), (50, 2723)]
值为11093的元素出现的位置: [(9, 2402), (14, 2026), (16, 651), (16, 726), (16, 1049), (16, 2407), (46, 3370), (55, 2355)]
值为60442的元素出现的位置: [(17, 77), (17, 1018), (17, 3390), (17, 3516)]
值为64199的元素出现的位置: [(18, 245), (18, 290), (18, 381), (18, 1328), (22, 3371), (23, 2143), (52, 2870)]
值为23780的元素出现的位置: [(19, 870), (19, 2339), (19, 3101), (19, 3351)]


值为18962的元素出现的位置: [(21, 860), (21, 871), (21, 1625), (21, 1749), (25, 28), (30, 3235)]
值为18929的元素出现的位置: [(1, 1492), (8, 3035), (12, 1745), (24, 210), (24, 1695), (24, 2041), (24, 3582)]
值为47425的元素出现的位置: [(21, 632), (25, 3083), (27, 3084), (28, 487), (28, 625), (28, 880), (28, 1937)]
值为10341的元素出现的位置: [(32, 327), (32, 781), (32, 1294), (32, 1354), (32, 2890), (32, 3233), (33, 2079)]
值为62270的元素出现的位置: [(33, 52), (33, 1907), (33, 1908), (33, 2736), (48, 2503)]
值为18288的元素出现的位置: [(35, 685), (35, 2155), (35, 2378), (35, 2645), (35, 3472)]
值为58413的元素出现的位置: [(10, 1111), (34, 3082), (36, 252), (36, 1671), (36, 2234), (36, 2984), (40, 1627), (42, 2578)]
值为35490的元素出现的位置: [(37, 339), (37, 1840), (37, 2279), (37, 2992), (47, 930), (50, 3177)]
值为65428的元素出现的位置: [(36, 2046), (39, 496), (39, 604), (39, 833), (39, 1361)]
值为30957的元素出现的位置: [(0, 2042), (5, 817), (8, 3392), (40, 636), (40, 848), (40, 1126), (40, 1684), (40, 2487)]
值为6047的元素出现的位置: [(32, 1952), (32, 2902), (39, 1015), (41, 164), (41, 1283), (41,

In [28]:
positions = []
for i, row in enumerate(cindex):
   for j, value in enumerate(row):
       if value == elements[2]:
           positions.append((i, j))
print(f"值为{elements[2]}的元素出现的位置:", positions)

# x=range()
# # 创建图像
# plt.figure(figsize=(10, 6))  # 设置图像大小

# # 绘制折线图
# plt.plot(x, y, label='sin(x)', color='b', linewidth=2)

# # 添加标题和标签
# plt.title('1D Plot of sin(x)', fontsize=16)
# plt.xlabel('X-axis', fontsize=14)
# plt.ylabel('Y-axis', fontsize=14)

# # 添加图例
# plt.legend()

# # 添加网格
# plt.grid(True, linestyle='--', alpha=0.7)

# # 显示图像
# plt.show()
# # # 输出结果
# print("值为60391的元素出现的位置:", positions)
        # for j in range(10):
        #     centroids[[elements[0]+j+1,elements[j+1]]]=centroids[[elements[j+1],elements[0]+j+1]]


# centroids = centroids.view(16384, 32).to(device=dev, dtype=torch.float32)
# # 创建 code 张量，确保在 dev 上并为 float32
# code = torch.zeros([16384, 4], dtype=torch.float32, device=dev)
# code = centroids[:, [0, 8, 16, 24]]

# centroids = centroids.view(65536, 8).to(dev)  # 修正拼写错误并准备张量
# centroids[[46680,36823]]=centroids[[36823,46680]]
# centroids[[46681,56983]]=centroids[[56983,46681]]
# centroids[[46682,63703]]=centroids[[63703,46682]]
# centroids[[46683,26519]]=centroids[[26519,46683]]
# centroids[[46352,53967]]=centroids[[53967,46352]]
# centroids = centroids.view(16384, 32).to(device=dev, dtype=torch.float32)
# # 创建 code 张量，确保在 dev 上并为 float32
# code = torch.zeros([16384, 4], dtype=torch.float32, device=dev)
# code = centroids[:, [0, 8, 16, 24]]
# positions = []
# for i, row in enumerate(cindex):
#     for j, value in enumerate(row):
#         if value == 63839:
#             positions.append((i, j))

# # 输出结果
# print("值为60391的元素出现的位置:", positions)

In [42]:
import torch
from collections import OrderedDict, defaultdict
from collections import deque, defaultdict
class cache_sim:
    def __init__(self, mode, linesize, cachesize, num_lin, codebook):
        self.linesize = linesize
        self.num_lin = num_lin
        self.cache = torch.full((num_lin, linesize), -1)
        self.codebook = codebook
        
        # 预建 codebook 值到行的快速映射 (值 -> 行索引)
        self.codebook_value_map = {}
        for row_idx, row in enumerate(codebook):
            for val in row:
                val = val.item()
                if val not in self.codebook_value_map:
                    self.codebook_value_map[val] = row_idx
        
        # 使用 OrderedDict 实现高效 LRU 机制
        self.lru = OrderedDict()
        
        # 值到缓存行的反向映射 (值 -> 所在行集合)
        self.value_to_lines = defaultdict(set)
        # 缓存行到值的反向映射 (行 -> 存储的值集合)
        self.line_to_values = [set() for _ in range(num_lin)]

    def maintain_record(self, line):
        """ 更新 LRU 记录，O(1) 时间复杂度 """
        if line in self.lru:
            self.lru.move_to_end(line)
        else:
            self.lru[line] = None

    def is_empty(self):
        """ 返回可替换的行或 -1 """
        if len(self.lru) < self.num_lin:
            return -1
        else:
            # 弹出 LRU 行并清理其反向映射
            lru_line, _ = self.lru.popitem(last=False)
            for val in self.line_to_values[lru_line]:
                self.value_to_lines[val].remove(lru_line)
                if not self.value_to_lines[val]:
                    del self.value_to_lines[val]
            self.line_to_values[lru_line].clear()
            return lru_line

    def load_data(self, data):
        """ 加载数据到缓存，向量化操作 """
        data_val = data.item()
        if data_val not in self.codebook_value_map:
            raise ValueError(f"Value {data_val} not in codebook")
        
        code_row = self.codebook[self.codebook_value_map[data_val]]
        target_line = self.is_empty()
        
        if target_line == -1:
            # 寻找未使用的行
            for line in range(self.num_lin):
                if line not in self.lru:
                    target_line = line
                    break
            # 如果所有行已使用，触发 LRU 替换
            if target_line == -1:
                target_line = self.is_empty()
        # print(target_line)
        # 更新缓存行数据
        self.cache[target_line] = code_row
        # 更新反向映射
        new_values = set(code_row.tolist())
        for val in new_values:
            self.value_to_lines[val].add(target_line)
        self.line_to_values[target_line] = new_values
        self.maintain_record(target_line)

    def sim(self, read_data):
        """ 批量查询优化，O(1) 时间复杂度 """
        data_val = read_data.item()
        
        # 
        if data_val in self.value_to_lines:
            # 取第一个关联行并更新 LRU
            line = next(iter(self.value_to_lines[data_val]))
            self.maintain_record(line)
            return True
        else:
            self.load_data(read_data)
            # print('not hit')
            return False

In [43]:
def run_simulation(code,recon_weight):
    # 将输入数据转换为向量化操作
    cache_l1=cache_sim(mode=None, linesize=4, cachesize=None, num_lin=256, codebook=code)
    cache_l2=cache_sim(mode=None, linesize=4, cachesize=None, num_lin=2048, codebook=code)
    recon_flatten = recon_weight.reshape(-1) #按行解码
    hit_cnt = torch.sum(torch.tensor([
        cache_l1.sim(val) for val in recon_flatten
    ], dtype=torch.int32))
    hit_cnt=hit_cnt.item()
    sum_cnt=len(recon_flatten)
    print(f"Hit Rate: {hit_cnt / sum_cnt:.6f}")
    hit_cnt = torch.sum(torch.tensor([
        cache_l2.sim(val) for val in recon_flatten
    ], dtype=torch.int32))
    hit_cnt=hit_cnt.item()
    sum_cnt=len(recon_flatten)
    print(f"Hit Rate: {hit_cnt / sum_cnt:.6f}")
    return None
run_simulation(code,recon_weight)


Hit Rate: 0.603347
Hit Rate: 0.998895
